In [1]:
%load_ext autoreload
%autoreload 2
from balatro import Card, Run_Manager, Deck_Manager
import pandas as pd

In [27]:
deck_m = Deck_Manager()
blind_m = Run_Manager()

In [41]:
blind_m.blind_tracker(deck_m)

--- Blind Goal:  300  ---
 *** Round  1 *** 
{'score': 96, 'hand_type': 'Two Pair', 'cards_played': 4, 'discards_used': 0}
Current Round Score: 96
{'score': 112, 'hand_type': 'Two Pair', 'cards_played': 4, 'discards_used': 1}
Current Round Score: 208
{'score': 108, 'hand_type': 'Two Pair', 'cards_played': 4, 'discards_used': 0}
Current Round Score: 316
**** You've reached the Blind Goal ****

Total Run Score: 316.0 

--- Blind Goal:  450  ---
 *** Round  2 *** 
{'score': 104, 'hand_type': 'Two Pair', 'cards_played': 4, 'discards_used': 1}
Current Round Score: 104
{'score': 92, 'hand_type': 'Two Pair', 'cards_played': 4, 'discards_used': 0}
Current Round Score: 196
{'score': 60, 'hand_type': 'Pair', 'cards_played': 2, 'discards_used': 2}
Current Round Score: 256
{'score': 60, 'hand_type': 'Pair', 'cards_played': 2, 'discards_used': 0}
Current Round Score: 316
**** You did not meet the Goal for the Blind ****
Total Run Score: 316.0 



In [13]:
deck_m.draw_cards_and_get_best_hand()

{'score': 124, 'hand_type': 'Two Pair', 'cards_played': 4, 'discards_used': 0}

In [14]:
deck_m.shuffle_deck()